In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn import tree
import plotly.express as px

In [2]:
pokerData = pd.read_csv('poker-hand-training-true.data')
pokerData.head()

,1,10,1.1,11,1.2,13,1.3,12,1.4,1.5,9
0,2,11,2,13,2,10,2,12,2,1,9
1,3,12,3,11,3,13,3,10,3,1,9
2,4,10,4,11,4,1,4,13,4,12,9
3,4,1,4,13,4,12,4,11,4,10,9
4,1,2,1,4,1,5,1,3,1,6,8


In [3]:


# Go from multi variable classification to binary by setting 2 pair, 3 of a kind and 4 of a kind to a 1, else 0

pokerData.loc[pokerData['9'] == 1, '9'] = 0
pokerData.loc[pokerData['9'] == 2, '9'] = 1
pokerData.loc[pokerData['9'] == 3, '9'] = 1
pokerData.loc[pokerData['9'] == 4, '9'] = 0
pokerData.loc[pokerData['9'] == 5, '9'] = 0
pokerData.loc[pokerData['9'] == 6, '9'] = 0
pokerData.loc[pokerData['9'] == 7, '9'] = 1
pokerData.loc[pokerData['9'] == 8, '9'] = 0
pokerData.loc[pokerData['9'] == 9, '9'] = 0

X = pokerData.iloc[:,0:9].values
y = pokerData.iloc[:,10].values
pokerData.head()


import plotly.express as px

In [4]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size= 20000, random_state=1738)


# RANDOM FORREST TRIAL ONE ON POKER DATASET

In [5]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = pokerData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                

    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   30.0s finished


In [6]:
TRIAL_1.cv_results_

{'mean_fit_time': array([0.39744115, 0.76966166, 1.54352708, 0.42286353, 0.85043106,
        1.69325609, 0.46549983, 0.89747162, 1.78083138, 0.48101311,
        0.93840699, 1.90443797, 0.40444803, 0.81940465, 1.60718203,
        0.46780214, 0.92609625, 1.81816359, 0.51013827, 0.96122656,
        1.93376322, 0.51634407, 1.01066937, 2.06187339, 0.39644084,
        0.76085424, 1.53552055, 0.42946901, 0.86354275, 1.75711079,
        0.46259828, 0.92859907, 1.79564352, 0.48341541, 0.94381151,
        2.10651207, 0.4072504 , 0.91218395, 1.81375933, 0.53496013,
        0.94691443, 1.82707076, 0.49752769, 1.00726676, 2.07978926,
        0.5117403 , 1.11435866, 2.21180196, 0.41555748, 0.82200689,
        1.59888811, 0.44027867, 0.88386011, 1.92615647, 0.51274085,
        0.99025145, 1.92045145, 0.49802818, 0.97103515, 1.8383811 ,
        0.39113603, 0.77506633, 1.57865748, 0.44738479, 0.90167556,
        1.79574442, 0.48811951, 0.95321999, 1.95067744, 0.51814542,
        1.00536437, 1.66553226]

In [7]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_neg_log_loss']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [8]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_f1_micro']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [9]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [10]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :0.9315
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best F1 hyperparameters :0.9315
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :0.93165
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL TWO ON POKER DATASET

In [11]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = pokerData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                

    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   27.4s finished


In [12]:
TRIAL_2.cv_results_

{'mean_fit_time': array([0.36971812, 0.73693366, 1.48257365, 0.42306352, 0.83271623,
        1.6503191 , 0.4539906 , 0.87965665, 1.80034847, 0.47210546,
        0.95352001, 1.89593053, 0.41175408, 0.80148897, 1.54612932,
        0.4426806 , 0.88536158, 1.87501225, 0.51153932, 1.11936274,
        2.10420985, 0.60912395, 1.12056308, 2.1672637 , 0.41555705,
        0.7948833 , 1.69175487, 0.43617535, 0.88726311, 1.86810651,
        0.50713592, 0.98895025, 1.85769725, 0.49102206, 0.94431257,
        1.88722305, 0.3997436 , 0.830514  , 1.58246059, 0.44538317,
        0.88476119, 1.74540105, 0.48611803, 0.95276208, 1.90663948,
        0.48761945, 0.98194456, 2.00362296, 0.39183726, 0.797686  ,
        1.5088973 , 0.42456508, 0.83991933, 1.66973615, 0.44878607,
        0.88766317, 1.77162352, 0.46209712, 0.91598768, 1.93646517,
        0.43367343, 0.78197241, 1.6409112 , 0.45088782, 0.97283597,
        1.9312604 , 0.50553479, 1.00896711, 1.98120427, 0.51814561,
        0.99455519, 1.68034554]

In [13]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_neg_log_loss']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [14]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_f1_micro']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [15]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [16]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :0.9314
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best F1 hyperparameters :0.9317
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :0.9315
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL THREE ON POKER DATASET

In [17]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = pokerData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   27.6s finished


In [18]:
TRIAL_3.cv_results_

{'mean_fit_time': array([0.42276368, 0.79017954, 1.63640747, 0.46379881, 0.90337691,
        1.79624457, 0.47280674, 0.94741478, 1.84268479, 0.48992095,
        0.93780642, 1.88051705, 0.40564852, 0.7938827 , 1.5955719 ,
        0.45168839, 0.90017443, 1.75330787, 0.47821116, 1.05420642,
        2.01613374, 0.51664419, 0.98845029, 2.02063756, 0.40374737,
        0.79438334, 1.54352756, 0.44378171, 0.87365127, 1.71817732,
        0.4877192 , 0.9403089 , 1.85679688, 0.49092197, 0.97473812,
        1.9199513 , 0.41355538, 0.86914749, 1.71967883, 0.49362459,
        0.93930745, 1.88031707, 0.50213203, 0.99465532, 2.05706902,
        0.55697927, 1.10434971, 2.11551933, 0.40705004, 0.77937021,
        1.60057664, 0.47741022, 0.90287628, 1.75711102, 0.49092236,
        0.95392051, 1.92355433, 0.47070479, 0.99255395, 1.9782011 ,
        0.43477387, 0.79768572, 1.63420544, 0.47080503, 0.92609653,
        1.77262392, 0.48141446, 0.94881601, 1.89653049, 0.50813684,
        0.98034301, 1.60718212]

In [19]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_neg_log_loss']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 400}

In [20]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_f1_micro']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [21]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 400}

In [22]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :0.93165
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best F1 hyperparameters :0.9315
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :0.9317


# RANDOM FOREST TRIAL FOUR ON POKER DATASET

In [23]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = pokerData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   28.4s finished


In [24]:
TRIAL_4.cv_results_

{'mean_fit_time': array([0.3864326 , 0.75965266, 1.56174245, 0.428268  , 0.8694478 ,
        1.7053669 , 0.45178828, 0.90517874, 1.81876431, 0.471205  ,
        0.94311099, 1.93755636, 0.43807621, 0.86234179, 1.69145503,
        0.45999575, 0.88746314, 1.78873801, 0.48431644, 0.97333694,
        1.97700019, 0.51424246, 0.99925933, 2.11101575, 0.42976995,
        0.86934752, 1.66744623, 0.48061333, 0.89306846, 1.74249864,
        0.48451695, 0.93700576, 1.84418612, 0.48862014, 0.97674017,
        1.91754918, 0.4231636 , 0.8286129 , 1.65862637, 0.46690173,
        0.91588793, 2.02474117, 0.5503737 , 1.09374108, 2.33320632,
        0.58710437, 1.12036328, 2.17767262, 0.47400727, 0.870048  ,
        1.69705939, 0.47010474, 0.98324518, 1.82026558, 0.49962964,
        1.00746636, 1.93926749, 0.51794538, 0.99335442, 1.9885098 ,
        0.44578362, 0.88766317, 1.64931803, 0.46940384, 0.90758071,
        1.88021684, 0.5141418 , 1.05610838, 1.99691734, 0.56418509,
        1.04790125, 1.65612435]

In [25]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_neg_log_loss']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [26]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_f1_micro']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 100}

In [27]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [28]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :0.9318
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best F1 hyperparameters :0.9316
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :0.93155
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL FIVE ON POKER  DATASET

In [29]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = pokerData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   27.9s finished


In [30]:
TRIAL_5.cv_results_

{'mean_fit_time': array([0.43357296, 0.80419111, 1.59033852, 0.42326393, 0.85833821,
        1.77122307, 0.46910334, 0.99845843, 1.8797164 , 0.48671861,
        0.91708846, 1.92795792, 0.41775937, 0.82751141, 1.67223802,
        0.44208064, 0.92089229, 1.83167539, 0.49422498, 0.96512971,
        1.89843235, 0.51133904, 1.00076079, 1.98140397, 0.38543129,
        0.77276459, 1.5691494 , 0.42656698, 0.86184115, 1.70096264,
        0.45709305, 0.91398602, 1.88712306, 0.47540879, 0.97603974,
        1.97870169, 0.42666698, 0.79698548, 1.70236425, 0.48661833,
        0.97684021, 1.97549415, 0.55247493, 1.14968843, 2.24271803,
        0.6151288 , 1.17581244, 2.19108438, 0.4728065 , 0.82470927,
        1.65542364, 0.44778533, 0.92659688, 1.8312747 , 0.499229  ,
        0.95612211, 1.86260185, 0.47460799, 0.94691458, 1.96028585,
        0.40244627, 0.84032226, 1.63430538, 0.4660006 , 0.90397696,
        1.82326765, 0.50303254, 0.99385505, 1.95958509, 0.51404195,
        1.01917644, 1.63150315]

In [31]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_neg_log_loss']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 400}

In [32]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_f1_micro']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [33]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [34]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :0.93155
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best F1 hyperparameters :0.9316
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :0.93195
------------------------------------------------------------------------------------------------
